In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment=None
pd.set_option('display.max_columns',60)
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
plt.rcParams['font.size']=24
from IPython.core.pylabtools import figsize


In [4]:
import seaborn as sns
sns.set(font_scale=2)
from sklearn.preprocessing import Imputer,MinMaxScaler


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
import lime
import lime.lime_tabular

In [9]:
# Read in data into dataframes 
train_features = pd.read_csv('data/training_features.csv')
test_features = pd.read_csv('data/testing_features.csv')
train_labels = pd.read_csv('data/training_labels.csv')
test_labels = pd.read_csv('data/testing_labels.csv')

In [11]:
imputer=Imputer(strategy='median')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [12]:
imputer.fit(train_features)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

In [13]:
X=imputer.transform(train_features)
X_test=imputer.transform(test_features)


In [14]:
X_test

array([[1.022000e+04, 5.729364e+06, 1.239100e+05, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.289100e+04, 4.954012e+06, 2.663160e+05, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [7.434000e+03, 3.533538e+06, 5.436500e+04, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       ...,
       [4.033000e+03, 4.952360e+06, 9.301500e+04, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [5.075000e+03, 4.407838e+06, 1.127000e+05, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [8.859000e+03, 4.040464e+06, 1.671080e+05, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00]])

In [15]:
y=np.array(train_labels).reshape((-1,))
y_test=np.array(test_labels).reshape((-1,))

In [16]:
y_test

array([13., 55., 87., ..., 83., 51., 96.])

In [17]:
def mae(y_true,y_pred):
    return np.mean(abs(y_true=y_pred))



In [19]:
model = GradientBoostingRegressor(loss='lad', max_depth=5, max_features=None,
                                  min_samples_leaf=6, min_samples_split=6, 
                                  n_estimators=800, random_state=42)

model.fit(X, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='lad', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=6,
             min_samples_split=6, min_weight_fraction_leaf=0.0,
             n_estimators=800, n_iter_no_change=None, presort='auto',
             random_state=42, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [23]:
#  Make predictions on the test set
model_pred = model.predict(X_test)

print('Final Model Performance on the test set: MAE = %0.4f' % mae(y_test, model_pred))

TypeError: abs() takes no keyword arguments

In [22]:
print('Final Model Performance on the test set: MAE = %0.4f' % mae(y_test, model_pred))

TypeError: abs() takes no keyword arguments